In [9]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/pSp_PrivacyEncoder')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
import cv2
import glob
import dlib

from tqdm import tqdm


In [11]:
! wget http://arunponnusamy.com/files/mmod_human_face_detector.dat

--2021-04-22 04:17:22--  http://arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving arunponnusamy.com (arunponnusamy.com)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to arunponnusamy.com (arunponnusamy.com)|185.199.109.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.arunponnusamy.com/files/mmod_human_face_detector.dat [following]
--2021-04-22 04:17:23--  https://www.arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving www.arunponnusamy.com (www.arunponnusamy.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.arunponnusamy.com (www.arunponnusamy.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729940 (713K) [application/octet-stream]
Saving to: ‘mmod_human_face_detector.dat.6’

mmod_human_face_det 100%[===================>] 712.83K  --.-KB/s    in 0.05s   

2021-04-22 04:17:23 (13.3 MB/s) - ‘mmod_human_face_de

In [18]:
# HOG/CNN
#model = "HOG"
model = "CNN"
#input_dir = './image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned'
input_dir = './image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred'

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP', '.tiff'
]
def is_image_file(filename):
  return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

images_path = []
for root, dirs, files in os.walk(input_dir):
  for fname in files:
    if is_image_file(fname):
      img_path = os.path.join(root, fname)
      images_path.append(img_path)

images_path = sorted(images_path)

print('Load {} images'.format(len(images_path)))


Load 988 images


In [19]:
class face_detection:
  def __init__(self, model):
    if model == 'CNN':
      self.detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    elif model == 'HOG':
      self.detector = dlib.get_frontal_face_detector()
    else:
      print('Could load corresponded detection model!')
      exit()

  def detection(self, images_path):
    try:
      img_num = 0
      face_num = 0

      with tqdm(images_path, total=len(images_path)) as t:
        for img_path in t:
          img = cv2.imread(img_path)
          img_num += 1
          
          face = self.detector(img, 1)
          face_num += len(face)
        return img_num, face_num

    except KeyboardInterrupt:
      t.close()
      raise


In [20]:
detector = face_detection(model=model)

img_num, face_num = detector.detection(images_path=images_path)
print('\nThe percentage of {} detected faces is {:.2f}'.format(model, face_num/img_num))


RuntimeError: ignored

In [9]:
img_path = './image_datasets/Privacy_Evaluation.png'
output_dir = './image_datasets'

img = cv2.imread(img_path)

cnn_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

faces_cnn = cnn_detector(img, 1)

print('{} faces are detected in this image'.format(len(faces_cnn)))

for face in faces_cnn:
  x = face.rect.left()
  y = face.rect.top()
  w = face.rect.right() - x
  h = face.rect.bottom() - y

  cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 2)
  
  cv2.imwrite(os.path.join(output_dir, 'cnn_face_detected.png'), img)


8 faces are detected in this image


In [11]:
img_path = './image_datasets/Privacy_Evaluation.png'
output_dir = './image_datasets'

img = cv2.imread(img_path)

hog_detector = dlib.get_frontal_face_detector()

faces_hog = hog_detector(img, 1)

print('{} faces are detected in this image'.format(len(faces_hog)))

for face in faces_hog:
  x = face.left()
  y = face.top()
  w = face.right() - x
  h = face.bottom() - y

  cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 2)
  
  cv2.imwrite(os.path.join(output_dir, 'hog_face_detected.png'), img)

7 faces are detected in this image
